In [3]:
import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import os

In [80]:
import numpy as np
import pandas as pd

In [1]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

C:\Wesley_Tao\6.Software\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [125]:
from nltk.tokenize import word_tokenize
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
#     print(tokens_re.search(tokens[0]))
    if lowercase:
        tokens =  [token if emoticon_re.search(token)  else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'a n', 'example', '!', ':D', 'http://example.com', '#NLP']


['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [137]:
from nltk.corpus import stopwords
import string
def remove_stop_words(tweet):
    out=tweet
    if(tweet):
        p=re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
        p2=re.compile(r'(?:@[\w_]+)')
        punctuation = list(string.punctuation)
        stop = stopwords.words('english') + punctuation + ['via',"\r",r'<[^>]+>',"RT","…","’","—","u"]+[r'(?:@[\w_]+)']
    #     print(stop)
        terms_stop = [term for term in preprocess(tweet) if p.match(term)==None and term not in stop and p2.match(term)==None]
    #     terms_stop = [term for term in preprocess(tweet) if term not in stop]
        out=" ".join(terms_stop)
    return(out)

# US news 

In [113]:
myfile=[]
for file in os.listdir("../data/News/US"):
    myfile.append("../data/News/US/"+file)
article_data_us=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        article_data_us.append(thisdoc)

In [114]:
len(article_data_us)

25

In [138]:
article_data_us=list(map(remove_stop_words,article_data_us))

In [141]:
article_data_us[1]

"Trump trade war China tells US defend national interests 24 March 2018 Share Facebook Share Twitter Share Messenger Share Email Share Image copyrightREUTERS Image caption President Trump announced plans impose tariffs 60 bn Chinese products China warned US defend interests trade Chinese state media says US President Donald Trump backed tariffs Chinese goods The comments came phone call China's vice-premier Liu He US Treasury Secretary Steven Mnuchin Mr Trump announced plans impose tariffs 60 bn ￡ 42.5 bn Chinese goods accusing China intellectual property theft The move rattled markets stoked fears trade war Mr Liu Chinese President Xi Jinping's top economic adviser told Mr Mnuchin Beijing ready defend national interests hoped sides remain rational work together China's official Xinhua news agency reported During telephone conversation thought highest-level contact two governments since Mr Trump announced tariffs Thursday Mr Liu also accused US violating international trade rules follo

In [139]:
def topic(list_document,num_topics):
    """list_document is a list of document"""
    num_topics=num_topics
    doc_clean = [clean(doc).split() for doc in list_document] 
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    mylist=[]
    for topic in range(num_topics):
        termlist=ldamodel.print_topics(num_topics=num_topics, num_words=10)[topic][1].split("+")
        mylist.append(list(map(lambda term:term.split("*"),termlist)))
    pandas_list=[]
    for topic_index in range(num_topics):
        this_topic=pd.DataFrame(mylist[topic_index],columns=["prob","key_words"])
        this_topic["topic"]=topic_index
        pandas_list.append(this_topic)
    LDA_result=pd.concat(pandas_list)
    return(LDA_result)

In [140]:
topic(article_data_us,2)

,prob,key_words,topic
0,0.020,"""china""",0
1,0.016,"""trade""",0
2,0.010,"""growth""",0
3,0.009,"""trump""",0
4,0.008,"""economy""",0
5,0.007,"""year""",0
6,0.007,"""said""",0
7,0.006,"""state""",0
8,0.006,"""war""",0
9,0.006,"""2018""",0


# China

In [132]:
myfile=[]
for file in os.listdir("../data/News/China"):
    myfile.append("../data/News/China/"+file)
article_data_china=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        article_data_china.append(thisdoc)

In [133]:
len(article_data_china)

32

In [143]:
article_data_china=list(map(remove_stop_words,article_data_china))

In [144]:
article_data_china[1]

"Trade protectionism never win US competitiveness experts By Zhang Niansheng Wu Lejun Hu Zexi Zhang Mengxu People's Daily 09 42, April 09, 2018 The short-sighted trade protectionism measures taken US helpless improve competitiveness global arena experts pointed Office US Trade Representative USTR Tuesday published proposed list Chinese goods subject additional 25 percent tariffs despite strong opposition China US business groups The proposed list covered approximately 1,300 products imported China based so-called Section 301 investigation alleged Chinese intellectual property technology transfer practices launched Trump administration August 2017 The restricted measures launched US mainly targeting technology industries aerospace information communication technology robotics machinery cited Made China 2025 strategy plan upgrade manufacturing sector excuse But experts found trade deficit US China blamed long-term discriminatory export controls China well strict control high-tech exports

In [142]:
topic(article_data_china,3)

,prob,key_words,topic
0,0.030,"""china""",0
1,0.022,"""trade""",0
2,0.021,"""u""",0
3,0.011,"""said""",0
4,0.010,"""war""",0
5,0.008,"""trump""",0
6,0.008,"""chinese""",0
7,0.008,"""”""",0
8,0.008,"""“""",0
9,0.008,"""2018""",0


### FOX

In [19]:
with open
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/FOX.txt")
doc1 = fh.read()
doc_complete = [doc1]

In [20]:
doc_clean = [clean(doc).split() for doc in doc_complete] 

In [21]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [22]:
Lda = gensim.models.ldamodel.LdaModel

In [23]:
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [25]:
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.023*"farmer" + 0.016*"trade" + 0.016*"china" + 0.012*"u" + 0.012*"trump" + 0.010*"said" + 0.010*"agriculture" + 0.008*"president" + 0.008*"fox" + 0.008*"many"'), (1, '0.003*"farmer" + 0.003*"china" + 0.003*"trade" + 0.003*"tech" + 0.003*"economic" + 0.003*"actively" + 0.003*"feel" + 0.003*"reentering" + 0.003*"operator" + 0.003*"carry"'), (2, '0.003*"farmer" + 0.003*"china" + 0.003*"trade" + 0.003*"trump" + 0.003*"“ambassador" + 0.003*"king" + 0.003*"kansa" + 0.003*"still" + 0.003*"advantage" + 0.003*"job"')]


### China Daily

In [26]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/ChinaDaily/10.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.031*"china" + 0.027*"u" + 0.023*"said" + 0.020*"trade" + 0.009*"chinese" + 0.009*"tariff" + 0.009*"soybean" + 0.007*"world" + 0.006*"lng" + 0.006*"would"'), (1, '0.001*"china" + 0.001*"u" + 0.001*"said" + 0.001*"trade" + 0.001*"soybean" + 0.001*"tariff" + 0.001*"world" + 0.001*"chinese" + 0.001*"would" + 0.001*"farmer"'), (2, '0.001*"china" + 0.001*"u" + 0.001*"said" + 0.001*"trade" + 0.001*"chinese" + 0.001*"world" + 0.001*"tariff" + 0.001*"soybean" + 0.001*"year" + 0.001*"lng"')]


### CNN

In [28]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/CNN/CNNALL.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.001*"trade" + 0.001*"china" + 0.001*"said" + 0.001*"u" + 0.001*"trump" + 0.001*"state" + 0.001*"tariff" + 0.001*"united" + 0.001*"chinese" + 0.001*"war"'), (1, '0.001*"china" + 0.001*"trade" + 0.001*"said" + 0.001*"u" + 0.001*"trump" + 0.001*"war" + 0.001*"state" + 0.001*"united" + 0.001*"tariff" + 0.001*"chinese"'), (2, '0.022*"china" + 0.022*"trade" + 0.013*"u" + 0.013*"said" + 0.011*"trump" + 0.010*"chinese" + 0.009*"state" + 0.009*"war" + 0.009*"tariff" + 0.008*"united"')]


### Renmin

In [29]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/Renmin/Renmin.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.025*"china" + 0.018*"u" + 0.016*"trade" + 0.012*"chinese" + 0.009*"said" + 0.007*"world" + 0.007*"people" + 0.006*"product" + 0.006*"measure" + 0.006*"international"'), (1, '0.001*"china" + 0.001*"u" + 0.001*"trade" + 0.001*"chinese" + 0.001*"world" + 0.001*"said" + 0.001*"people" + 0.001*"product" + 0.001*"war" + 0.001*"international"'), (2, '0.001*"china" + 0.001*"u" + 0.001*"trade" + 0.001*"said" + 0.001*"chinese" + 0.001*"people" + 0.001*"world" + 0.001*"china’s" + 0.001*"product" + 0.001*"international"')]


### SCMP

In [31]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/SCMP/SCMP.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.025*"china" + 0.024*"u" + 0.017*"chinese" + 0.016*"trade" + 0.015*"said" + 0.013*"2018" + 0.009*"canada" + 0.009*"photo" + 0.008*"tariff" + 0.008*"import"'), (1, '0.003*"u" + 0.003*"chinese" + 0.003*"china" + 0.003*"said" + 0.003*"trade" + 0.003*"official" + 0.003*"edition" + 0.003*"unilateral" + 0.003*"hope" + 0.003*"unfair"'), (2, '0.003*"china" + 0.003*"u" + 0.003*"chinese" + 0.003*"trade" + 0.003*"2018" + 0.003*"said" + 0.003*"photo" + 0.003*"apr" + 0.003*"war" + 0.003*"canada"')]


### SCM

In [34]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/SOUTHCHINAMORNING.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.002*"u" + 0.002*"trade" + 0.002*"china" + 0.002*"bank" + 0.002*"company" + 0.002*"bloomberg" + 0.002*"three" + 0.002*"war" + 0.002*"2018" + 0.002*"tariff"'), (1, '0.020*"china" + 0.020*"trade" + 0.018*"u" + 0.012*"bloomberg" + 0.011*"bank" + 0.010*"company" + 0.010*"new" + 0.008*"would" + 0.008*"war" + 0.008*"chinese"'), (2, '0.002*"trade" + 0.002*"china" + 0.002*"u" + 0.002*"new" + 0.002*"bank" + 0.002*"bloomberg" + 0.002*"would" + 0.002*"chinese" + 0.002*"trump’s" + 0.002*"tariff"')]
